## Loading the necessary libraries and packages

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model #type: ignore
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from functools import partial
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import math

In [3]:
composite_images = np.load('composite_images.npz')
images_path = 'composite_images.npz'
# Load precipitation data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:

train_df['event_id'] = train_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
train_df['event_idx'] = train_df.groupby('event_id', sort=False).ngroup()
test_df['event_id'] = test_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
test_df['event_idx'] = test_df.groupby('event_id', sort=False).ngroup()

train_df['event_t'] = train_df.groupby('event_id').cumcount()
test_df['event_t'] = test_df.groupby('event_id').cumcount()

print(train_df.head())
print(test_df.head())

          event_id  precipitation  label  event_idx  event_t
0  id_spictby0jfsb       0.000000      0          0        0
1  id_spictby0jfsb       0.095438      0          0        1
2  id_spictby0jfsb       1.949560      0          0        2
3  id_spictby0jfsb       3.232160      0          0        3
4  id_spictby0jfsb       0.000000      0          0        4
          event_id  precipitation  event_idx  event_t
0  id_j7b6sokflo4k        0.00000          0        0
1  id_j7b6sokflo4k        3.01864          0        1
2  id_j7b6sokflo4k        0.00000          0        2
3  id_j7b6sokflo4k       16.61520          0        3
4  id_j7b6sokflo4k        2.56706          0        4


In [5]:
# Constants for image preprocessing
BAND_NAMES = ('B2', 'B3', 'B4', 'B8', 'B11', 'slope')
H, W, NUM_CHANNELS = IMG_DIM = (128, 128, len(BAND_NAMES))
_MAX_INT = np.iinfo(np.uint16).max

def decode_slope(x: np.ndarray) -> np.ndarray:
    # Convert 16-bit discretized slope to float32 radians
    return (x / _MAX_INT * (math.pi / 2.0)).astype(np.float32)

def normalize(x: np.ndarray, mean: int, std: int) -> np.ndarray:
    return (x - mean) / std

rough_S2_normalize = partial(normalize, mean=1250, std=500)

def preprocess_image(x: np.ndarray) -> np.ndarray:
    return np.concatenate([
        rough_S2_normalize(x[..., :-1].astype(np.float32)),
        decode_slope(x[..., -1:]),
    ], axis=-1, dtype=np.float32)


In [6]:
# Load composite images
with np.load('composite_images.npz') as data:
    print("Available event IDs in composite_images.npz:", data.keys())
    composite_images = {event_id: data[event_id] for event_id in data.keys()}  # Dictionary mapping event_ids to image arrays

# Preprocess data and images
def preprocess_data_and_images(data_df, composite_images):
    event_ids = data_df['event_id'].unique()
    timeseries = []
    labels = []
    images = []

    for event_id in tqdm(event_ids, desc="Processing data"):
        event_data = data_df[data_df['event_id'] == event_id]
        timeseries.append(event_data['precipitation'].values)  # Shape: (730,)
        if 'label' in event_data.columns:
            labels.append(event_data['label'].values)  # Shape: (730,)
        images.append(preprocess_image(composite_images[event_id]))  # Shape: (128, 128, 6)

    timeseries = np.array(timeseries)
    labels = np.array(labels) if labels else None
    images = np.stack(images, axis=0)

    return timeseries, labels, images


Available event IDs in composite_images.npz: KeysView(NpzFile 'composite_images.npz' with keys: id_rhg5w8vmv3ny, id_rua8ey2jczl0, id_073l04ir88sn, id_wmkfqw7iwjmu, id_heri806er7xw...)


In [7]:
def create_image_encoder(input_shape=(128, 128, 6)):
    """"
    Creates and improved CNN based image encoder for processing satellite images
    """
    inputs = layers.Input(shape=input_shape)

    # add batch normalization at input
    x = layers.BatchNormalization()(inputs)

    # CNN blocks with residual connections
    def conv_block(x, filters, kernel_size = 3):
        skip=x
        x = layers.Conv2D(filters, kernel_size, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters, kernel_size, padding='same')(x)
        x = layers.BatchNormalization()(x)

        # add residual connection if shapes are the same

        if skip.shape[-1] == filters:
            x = layers.add([x, skip])
        x = layers.Activation('relu')(x)
        return x
    # CNN architecture with residual blocks

    x = conv_block(x, 32)
    x = layers.MaxPooling2D((2,2))(x)
    x = conv_block(x, 64)
    x = layers.MaxPooling2D((2,2))(x)
    x = conv_block(x, 128)
    x = layers.MaxPooling2D((2,2))(x)

    return Model(inputs=inputs, outputs = x, name = 'image_encoder')




In [8]:
def create_flood_prediction_model():
    """
    Creates an enhanced model with attention mechanisms and a better handling of temporaral data
    """
    image_input = layers.Input(shape=(128, 128, 6), name='image_input')
    image_encoder = create_image_encoder()
    image_features = image_encoder(image_input)
    image_features = layers.Flatten()(image_features)

    # Precititation input with batch normalization
    precip_input = layers.Input(shape=(730, 1), name = 'precip_input')
    precip_normalized = layers.BatchNormalization()(precip_input)

    # Repeat image_features for each timestep
    image_features_repeated = layers.RepeatVector(730)(image_features)

    #combine features with attention mechanism
    combined = layers.Concatenate(axis=-1)([precip_normalized, image_features_repeated])

    # Attention mechanism
    attention = layers.Dense(1, activation='tanh')(combined)
    attention_weights = layers.Softmax(axis=1)(attention)
    combined = layers.Multiply()([combined, attention_weights])

    #Bidirectional LSTM layers with skip connectins
    lstm1 = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(combined)
    lstm1 = layers.Dropout(0.3)(lstm1)

    lstm2 = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(lstm1)
    lstm2 = layers.Dropout(0.3)(lstm2)

    # combine lstm outputs
    lstm_combined = layers.Concatenate()([lstm1, lstm2])

    # output layer with focal consideration
    outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(lstm_combined)

    return Model(inputs=[image_input, precip_input], outputs=outputs)


In [9]:
def focal_loss(gamma=2., alpha=.25):
    """
    Focal loss for handling class imbalance
    """
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -tf.reduce_mean(alpha * tf.pow(1. - pt_1, gamma) * tf.math.log(pt_1 + 1e-7) +
                             (1-alpha) * tf.pow(pt_0, gamma) * tf.math.log(1. - pt_0 + 1e-7))
    return focal_loss_fixed


In [10]:
def train_model(train_df, test_df, composite_images):
    """
    Main training fuction with improved data handling
    """
    # preprocess_data
    train_timeseries, train_labels, train_images = preprocess_data_and_images(train_df, composite_images)
    test_timeseries, _, test_images = preprocess_data_and_images(test_df, composite_images)

    # scale the timeseries data 
    scaler = StandardScaler()
    train_timeseries = scaler.fit_transform(train_timeseries.reshape(-1, 1)).reshape(train_timeseries.shape)
    test_timeseries = scaler.transform(test_timeseries.reshape(-1, 1)).reshape(test_timeseries.shape)

    # add chanel dimension to thetimeseries data
    train_timeseries = np.expand_dims(train_timeseries, axis=-1)
    test_timeseries = np.expand_dims(test_timeseries, axis=-1)

    # split trainiing data
    train_idx, val_idx = train_test_split(np.arange(len(train_images)), test_size = 0.2, random_state = 42, stratify = train_labels.max(axis=1))

    # prepare the validation data
    # Prepare validation data
    val_images = train_images[val_idx]
    val_timeseries = train_timeseries[val_idx]
    val_labels = train_labels[val_idx]
    
    # Update training data
    train_images = train_images[train_idx]
    train_timeseries = train_timeseries[train_idx]
    train_labels = train_labels[train_idx]

    # Compute class weights
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_labels.flatten()),
        y=train_labels.flatten()
    )
    class_weight_dict = dict(enumerate(class_weights))

     # Create and compile model
    model = create_flood_prediction_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=focal_loss(gamma=2.0, alpha=0.25),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.AUC(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ]
    )

    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_auc',
            patience=10,
            restore_best_weights=True,
            mode='max'
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_auc',
            factor=0.5,
            patience=5,
            mode='max'
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.keras',
            monitor='val_auc',
            save_best_only=True,
            mode='max'
        )
    ]

     # Train model
    history = model.fit(
        [train_images, train_timeseries],
        train_labels,
        validation_data=([val_images, val_timeseries], val_labels),
        epochs=100,
        batch_size=8,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        shuffle=True
    )
    
    return model, history, scaler

In [11]:
def make_predictions(model, test_timeseries, test_images):
    """
    Make predictions on the test data
    """
    predictions = model.predict([test_images, test_timeseries])
    return predictions

In [12]:
test_timeseries, _, test_images = preprocess_data_and_images(test_df, composite_images)


Processing data: 100%|██████████| 224/224 [00:07<00:00, 31.47it/s]


In [ ]:
# Train model
model, history, scaler = train_model(train_df, test_df, composite_images)
# Make predictions
#predictions = make_predictions(model, test_timeseries, test_images)

In [14]:
sample_submission = pd.read_csv('SampleSubmission (2).csv')
sample_submission.head()

,event_id,label
0,id_j7b6sokflo4k_X_0,0
1,id_j7b6sokflo4k_X_1,0
2,id_j7b6sokflo4k_X_2,0
3,id_j7b6sokflo4k_X_3,0
4,id_j7b6sokflo4k_X_4,0


In [18]:
sample_submission['label'] = pred.flatten()
sample_submission.head()

,ID,Target
0,id_j7b6sokflo4k_X_0,0.000156
1,id_j7b6sokflo4k_X_1,0.000328
2,id_j7b6sokflo4k_X_2,0.000156
3,id_j7b6sokflo4k_X_3,0.003241
4,id_j7b6sokflo4k_X_4,0.000295


In [15]:
sample_submission.to_csv("submission_201.csv", index=False)
